In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [2]:
data = pd.read_csv('kickstarter.csv', parse_dates=['launched'])
data.head(3)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00


In [3]:
data = data.assign(hour=data.launched.dt.hour,
                  day=data.launched.dt.day,
                  month=data.launched.dt.month,
                  year=data.launched.dt.year)
data.shape

(378661, 19)

In [4]:
data = data.query("state != 'live'")
data.shape

(375862, 19)

In [5]:
data = data.assign(outcome=(data.state == 'successful').astype(int))
data.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,hour,day,month,year,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,12,11,8,2015,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,4,2,9,2017,0


In [6]:
cat_features = ['category', 'currency', 'country']

In [7]:
sub_1 = data[['hour', 'day', 'month', 'year', 'goal', 'outcome']]

In [8]:
encoder = LabelEncoder()

In [9]:
encoded = data[cat_features].apply(encoder.fit_transform)
encoded.head()

,category,currency,country
0,108,5,9
1,93,13,22
2,93,13,22
3,90,13,22
4,55,13,22


In [10]:
X = pd.concat([sub_1, encoded], axis=1)
X.head()

,hour,day,month,year,goal,outcome,category,currency,country
0,12,11,8,2015,1000.0,0,108,5,9
1,4,2,9,2017,30000.0,0,93,13,22
2,0,12,1,2013,45000.0,0,93,13,22
3,3,17,3,2012,5000.0,0,90,13,22
4,8,4,7,2015,19500.0,0,55,13,22


In [11]:
X.shape

(375862, 9)

In [12]:
valid_fraction = 0.1

valid_size = int(len(X) * valid_fraction)

train = X[: -2 * valid_size]

valid = X[-2 * valid_size : -valid_size]

test = X[-valid_size:]

In [13]:
train.shape

(300690, 9)

In [14]:
test.shape

(37586, 9)

In [15]:
features = train.columns.drop(['outcome'])
features

Index(['hour', 'day', 'month', 'year', 'goal', 'category', 'currency',
       'country'],
      dtype='object')

In [16]:
dtrain = lgb.Dataset(train[features], train['outcome'])
dvalid = lgb.Dataset(valid[features], valid['outcome'])

dtrain

In [17]:
param = {'num_leaves': 65, 'objective': 'binary', 'metrics': 'auc'}

num_rounds = 1000

In [18]:
model = lgb.train(param, dtrain, num_rounds, valid_sets=[dvalid], early_stopping_rounds=5)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011627 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 528
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
[1]	valid_0's auc: 0.694476
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.69772
[3]	valid_0's auc: 0.700705
[4]	valid_0's auc: 0.702191
[5]	valid_0's auc: 0.706432
[6]	valid_0's auc: 0.708326
[7]	valid_0's auc: 0.709922
[8]	valid_0's auc: 0.71056
[9]	valid_0's auc: 0.71211
[10]	valid_0's auc: 0.713593
[11]	valid_0's auc: 0.71438
[12]	valid_0's auc: 0.715205
[13]	valid_0's auc: 0.717576
[14]	valid_0's

In [19]:
preds = model.predict(test[features])
score = metrics.roc_auc_score(test['outcome'], preds)
score

0.7464868776890301